# Scheduling Problem
#### The single machine total weighted tardiness problem (SMTWTP)
* we have a single machine that can handle only one job at a time
* there is an N number of jobs (or tasks) to be processed without interruption on the machine
* Each job i ∈ N requires:
    * an integer processing time $P_i$
    * a positive weight $W_i$ indicates the importance of the job
    * a due date $d_i$
    * a completion time of job i as $C_i$
    * the tardiness of the job can be calculated as Ti = max{Ci − di, 0}
    
    
### Objective
The objective is to order the N jobs in a way that **minimizes the total weighted tardiness** of the whole process.

* Tasks with same due date and different weight (same processing time)
* Tasks with different due dates and same weight (same processing time)

i.e., $min\sum Wi Ti$, notice that if a job has a higher weight the penalty of tardiness will be higher.

In [4]:
import pandas as pd
import random as rd
from itertools import combinations
import math


def input_data(Path):
    '''Takes the path of the excel file of the SMTWTP instances.
    Returns a dict of jobs number as Key and weight, processing time (hours) and due date (hours) as values.
    '''
    return pd.read_excel(Path, names=['Job', 'weight', "processing_time", "due_date"],
                             index_col=0).to_dict('index')

instance_dict = input_data('Instance_10.xlsx')
instance_dict

{1: {'weight': 0.5, 'processing_time': 1.5, 'due_date': 4},
 2: {'weight': 0.8, 'processing_time': 2.2, 'due_date': 4},
 3: {'weight': 0.6, 'processing_time': 0.5, 'due_date': 4},
 4: {'weight': 0.4, 'processing_time': 1.6, 'due_date': 6},
 5: {'weight': 0.4, 'processing_time': 3.5, 'due_date': 6},
 6: {'weight': 0.1, 'processing_time': 2.1, 'due_date': 6},
 7: {'weight': 0.2, 'processing_time': 2.5, 'due_date': 8},
 8: {'weight': 0.5, 'processing_time': 0.6, 'due_date': 8},
 9: {'weight': 0.3, 'processing_time': 3.2, 'due_date': 8},
 10: {'weight': 0.6, 'processing_time': 5.2, 'due_date': 8}}

In [10]:
def Objfun(instance_dict, solution, show = False):
    '''Takes a set of scheduled jobs, dict (input data)
    Return the objective function value of the solution
    '''
    dict = instance_dict
    t = 0   #starting time
    objfun_value = 0
    for job in solution:
        C_i = t + dict[job]["processing_time"]  # Completion time
        d_i = dict[job]["due_date"]   # due date of the job
        T_i = max(0, C_i - d_i)    #tardiness for the job
        W_i = dict[job]["weight"]  # job's weight

        objfun_value +=  W_i * T_i
        t = C_i
    if show == True:
        print("\n","#"*4, 
              "The Objective function value for {} solution schedule is: {}".format(solution ,objfun_value),"#"*4)
    return objfun_value

In [12]:
solution_1 = [1,2,5,6,8,9,10,3,4,7]
solution_2 = [2,3,5,10,6,8,9,4,7,1]

Objfun(instance_dict, solution_1, show=True)
Objfun(instance_dict,solution_2, show=True)


 #### The Objective function value for [1, 2, 5, 6, 8, 9, 10, 3, 4, 7] solution schedule is: 27.090000000000003 ####

 #### The Objective function value for [2, 3, 5, 10, 6, 8, 9, 4, 7, 1] solution schedule is: 26.0 ####


26.0

### Step 0: 
The initial step is to create an initial solution so the algorithm can iterate over it and find a better one. 

The initial solution can be seen as the starting point of the algorithm, in most cases, this initial solution is assigned randomly.

In [18]:
def get_InitialSolution(instance_dict, seed, show=False):
    n_jobs = len(instance_dict) # Number of jobs
    # Producing a random schedule of jobs
    initial_solution = list(range(1, n_jobs+1))
    rd.seed(seed)
    rd.shuffle(initial_solution)
    if show == True:
        print("initial Random Solution: {}".format(initial_solution))
    return initial_solution

In [19]:
get_InitialSolution(instance_dict, seed=42, show=True)

initial Random Solution: [8, 4, 3, 9, 6, 7, 10, 5, 1, 2]


[8, 4, 3, 9, 6, 7, 10, 5, 1, 2]

### Step 1: 
Now we need to create the list of candidate solutions from the current solution 𝕊 (initial solution in iteration 0), we call these solutions neighbors or the neighborhood of 𝕊. 


Let’s assume that the current solution in SMTWTP is 𝕊 = [3, 8, 10, 4, 1, 6, 2, 5, 9, 7] and we defined our neighborhood function as a swap move N(𝕊, Swap), i.e., replace the order of two jobs, thus one neighborhood solution would be [8, 3, 10, 4, 1, 6, 2, 5, 9, 7] where jobs 8 and 3 are swapped another would be [8, 3, 5, 4, 1, 6, 2, 10, 9, 7] swapping 5 and 10.

In [30]:
def SwapMove(solution, i ,j):
    '''Takes a list (solution)
    returns a new neighbor solution with i, j swapped
    '''
    solution = solution.copy()
    # job index in the solution:
    i_index = solution.index(i)
    j_index = solution.index(j)
    #Swap
    solution[i_index], solution[j_index] = solution[j_index], solution[i_index]
    return solution

initial_solution = get_InitialSolution(instance_dict, 2021, show=True);
print('Neighbor Solution:       {}'.format(SwapMove(initial_solution, 10, 5)))

initial Random Solution: [3, 8, 10, 4, 1, 6, 2, 5, 9, 7]
Neighbor Solution:       [3, 8, 5, 4, 1, 6, 2, 10, 9, 7]


### Step 2: 
From the neighborhood solutions list created in step 1, we choose the best admissible (Non-tabu or meets aspiration criteria) solution.

##### Tabu List

This is the list TS uses to record the recent solutions and prevents them to reoccur for a specified number of iterations, which helps the search to move away from previously visited solutions thus performs more extensive exploration.

##### Tabu Tenure
This defines the size of the Tabu list, i.e., for how many iterations a solution component is kept as Tabu. 
for example, the Tabu Tenure is 2, in iteration-1 the Tabu list is empty, Tabu list= {}, 
let’s assume that the best admissible solution found in iteration-1 is [8, 3, 5, 4, 1, 6, 2, 10, 9, 7] thus Tabu list becomes {[8, 3, 5, 4, 1, 6, 2, 10, 9, 7]}, in iteration-2, the best solution is still [8, 3, 5, 4, 1, 6, 2, 10, 9, 7], however, it’s not admissible because it’s Tabu, so we take the next best solution, let assume it’s [8, 3, 5, 4, 1, 6, 2, 10, 7,9], Tabu list now equals to {[8, 3, 5, 4, 1, 6, 2, 10, 9, 7], [8, 3, 5, 4, 1, 6, 2, 10, 7,9]}. In iteration-3, [8, 3, 5, 4, 1, 6, 2, 10, 9, 7] solution leaves the Tabu list, and the next best admissible solution goes into the list (Because the list size is 2), and so on. 


##### Aspiration Criteria
If the solution found in the current iteration has a value (objective function value) better than the currently-known best solution’s value, but the move (like don’t swap job 3 with 1) is Tabu, we can use the Aspiration Criteria to override the tabu state of this move, thereby including the otherwise-excluded solution in the allowed set.

### Step 3: 
Check the defined stopping criteria, this can be the max number of iterations reached or the time of running, if the stopping criteria are not met, go to step 4, if the stopping criteria are met terminate and return the best solution.

### Step 4: 
Update Tabu list, Aspiration Criteria, and go to Step 1

In [23]:
def get_tabuestructure(instance_dict):
    '''Takes a dict (input data)
    Returns a dict of tabu attributes(pair of jobs that are swapped) as keys and [tabu_time, MoveValue]
    '''
    dict = {}
    for swap in combinations(instance_dict.keys(), 2):
        dict[swap] = {'tabu_time': 0, 'MoveValue': 0}
    return dict

In [38]:
get_tabuestructure(instance_dict)

{(1, 2): {'tabu_time': 0, 'MoveValue': 0},
 (1, 3): {'tabu_time': 0, 'MoveValue': 0},
 (1, 4): {'tabu_time': 0, 'MoveValue': 0},
 (1, 5): {'tabu_time': 0, 'MoveValue': 0},
 (1, 6): {'tabu_time': 0, 'MoveValue': 0},
 (1, 7): {'tabu_time': 0, 'MoveValue': 0},
 (1, 8): {'tabu_time': 0, 'MoveValue': 0},
 (1, 9): {'tabu_time': 0, 'MoveValue': 0},
 (1, 10): {'tabu_time': 0, 'MoveValue': 0},
 (2, 3): {'tabu_time': 0, 'MoveValue': 0},
 (2, 4): {'tabu_time': 0, 'MoveValue': 0},
 (2, 5): {'tabu_time': 0, 'MoveValue': 0},
 (2, 6): {'tabu_time': 0, 'MoveValue': 0},
 (2, 7): {'tabu_time': 0, 'MoveValue': 0},
 (2, 8): {'tabu_time': 0, 'MoveValue': 0},
 (2, 9): {'tabu_time': 0, 'MoveValue': 0},
 (2, 10): {'tabu_time': 0, 'MoveValue': 0},
 (3, 4): {'tabu_time': 0, 'MoveValue': 0},
 (3, 5): {'tabu_time': 0, 'MoveValue': 0},
 (3, 6): {'tabu_time': 0, 'MoveValue': 0},
 (3, 7): {'tabu_time': 0, 'MoveValue': 0},
 (3, 8): {'tabu_time': 0, 'MoveValue': 0},
 (3, 9): {'tabu_time': 0, 'MoveValue': 0},
 (3, 10):

In [39]:
def TSearch(instance_dict, tabu_tenure):
    '''The implementation Tabu search algorithm with short-term memory and pair_swap as Tabu attribute.
    '''
    # Parameters:
    tenure = tabu_tenure
    tabu_structure = get_tabuestructure(instance_dict)  # Initialize the data structures
    best_solution = get_InitialSolution(instance_dict, 42)
    best_objvalue = Objfun(instance_dict, best_solution)
    current_solution = get_InitialSolution(instance_dict, 42)
    current_objvalue = Objfun(instance_dict, best_solution)

    print("#"*30, "Short-term memory TS with Tabu Tenure: {}\nInitial Solution: {}, Initial Objvalue: {}".format(
        tenure, current_solution, current_objvalue), "#"*30, sep='\n\n')
    iter = 1
    Terminate = 0
    while Terminate < 100:
        print('\n\n### iter {}###  Current_Objvalue: {}, Best_Objvalue: {}'.format(iter, current_objvalue,
                                                                                best_objvalue))
        # Searching the whole neighborhood of the current solution:
        for move in tabu_structure:
            candidate_solution = SwapMove(current_solution, move[0], move[1])
            candidate_objvalue = Objfun(instance_dict, candidate_solution)
            tabu_structure[move]['MoveValue'] = candidate_objvalue

        # Admissible move
        while True:
            # select the move with the lowest ObjValue in the neighborhood (minimization)
            best_move = min(tabu_structure, key =lambda x: tabu_structure[x]['MoveValue'])
            MoveValue = tabu_structure[best_move]["MoveValue"]
            tabu_time = tabu_structure[best_move]["tabu_time"]
            # Not Tabu
            if tabu_time < iter:
                # make the move
                current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                current_objvalue = Objfun(instance_dict, current_solution)
                # Best Improving move
                if MoveValue < best_objvalue:
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    print("   best_move: {}, Objvalue: {} => Best Improving => Admissible".format(best_move,
                                                                                                  current_objvalue))
                    Terminate = 0
                else:
                    print("   ##Termination: {}## best_move: {}, Objvalue: {} => Least non-improving => "
                          "Admissible".format(Terminate,best_move, current_objvalue))
                    Terminate += 1
                # update tabu_time for the move
                tabu_structure[best_move]['tabu_time'] = iter + tenure
                iter += 1
                break
            # If tabu
            else:
                # Aspiration
                if MoveValue < best_objvalue:
                    # make the move
                    current_solution = SwapMove(current_solution, best_move[0], best_move[1])
                    current_objvalue = Objfun(instance_dict, current_solution)
                    best_solution = current_solution
                    best_objvalue = current_objvalue
                    print("   best_move: {}, Objvalue: {} => Aspiration => Admissible".format(best_move,
                                                                                                  current_objvalue))
                    Terminate = 0
                    iter += 1
                    break
                else:
                    tabu_structure[best_move]["MoveValue"] = float('inf')
                    print("   best_move: {}, Objvalue: {} => Tabu => Inadmissible".format(best_move,
                                                                                          current_objvalue))
                    continue
    print('#'*50 , "Performed iterations: {}".format(iter), "Best found Solution: {} , Objvalue: {}".format(best_solution,best_objvalue), sep="\n")
    return tabu_structure, best_solution, best_objvalue

In [40]:

test = TSearch(instance_dict, tabu_tenure=3)

##############################

Short-term memory TS with Tabu Tenure: 3
Initial Solution: [8, 4, 3, 9, 6, 7, 10, 5, 1, 2], Initial Objvalue: 34.07

##############################


### iter 1###  Current_Objvalue: 34.07, Best_Objvalue: 34.07
   best_move: (2, 9), Objvalue: 22.339999999999996 => Best Improving => Admissible


### iter 2###  Current_Objvalue: 22.339999999999996, Best_Objvalue: 22.339999999999996
   best_move: (1, 6), Objvalue: 16.240000000000002 => Best Improving => Admissible


### iter 3###  Current_Objvalue: 16.240000000000002, Best_Objvalue: 16.240000000000002
   best_move: (1, 4), Objvalue: 15.120000000000003 => Best Improving => Admissible


### iter 4###  Current_Objvalue: 15.120000000000003, Best_Objvalue: 15.120000000000003
   best_move: (5, 7), Objvalue: 14.380000000000004 => Best Improving => Admissible


### iter 5###  Current_Objvalue: 14.380000000000004, Best_Objvalue: 14.380000000000004
   best_move: (2, 8), Objvalue: 13.860000000000003 => Best Improving 